In [1]:
track = "GSU24AI03-SU24AI21/tracked-url-video"
download = "GSU24AI03-SU24AI21/downloaded-vietnamese-video"
speaker = "GSU24AI03-SU24AI21/detected-speaker-clip"
crop = "GSU24AI03-SU24AI21/cropped-mouth-clip"
vndetect = "GSU24AI03-SU24AI21/detected-vietnamese-clip"
trans = "GSU24AI03-SU24AI21/transcribed-vietnamese-audio"
vnav = "GSU24AI03-SU24AI21/vietnamese-av"

In [6]:
import datasets
dataset = datasets.load_dataset(
    vnav,
    name="all",
    trust_remote_code=True,
    cache_dir="../../../data/external/vietnamese-av",
)